<a href="https://colab.research.google.com/github/carlosdgerez/machine_learning/blob/main/daysResponses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Importing the 2 sets. one for the model, and the second for final testing. 

import pandas as pd

campaign = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bank.csv')
last_campaign = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/bank_holdout_test.csv')

In [3]:
# slice of responses for marketing the customer was contacted at least once
contacted = campaign[campaign.pdays != 999]
contacted.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1515 entries, 24108 to 41182
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             1515 non-null   int64  
 1   job             1515 non-null   object 
 2   marital         1515 non-null   object 
 3   education       1515 non-null   object 
 4   default         1515 non-null   object 
 5   housing         1515 non-null   object 
 6   loan            1515 non-null   object 
 7   contact         1515 non-null   object 
 8   month           1515 non-null   object 
 9   day_of_week     1515 non-null   object 
 10  campaign        1515 non-null   int64  
 11  pdays           1515 non-null   int64  
 12  previous        1515 non-null   int64  
 13  poutcome        1515 non-null   object 
 14  emp.var.rate    1515 non-null   float64
 15  cons.price.idx  1515 non-null   float64
 16  cons.conf.idx   1515 non-null   float64
 17  euribor3m       1515 non-nul